**"Corrupted document" challenge solution - Lorant Zsarnowszky**

Importing the necessary libraries

In [1]:
import os
import pandas as pd
import numpy as np
import nltk
from wordfreq import word_frequency

Importing the corrupted text file

In [2]:
os.chdir(r'c:\Lori\MUNKA\world_quant\wqp-challenge')

In [3]:
df_corr = pd.read_csv('corrupted_tokens.txt', sep=" ", header=None, names=["Original"])

In [4]:
df_corr.sample(10)

,Original
10257,w#e#
27621,t#e
9675,a#d
723,t#
8933,A#d
27463,o#e#
2562,t#e
7696,f#v#
17266,f#r#i#h#d
25643,t#


In [5]:
# checking for null values
df_corr.isnull().sum()

Original    0
dtype: int64

Creating word list

In [6]:
from nltk.corpus import words
word_list = words.words()

Coding the trie for the task

In [7]:
from __future__ import print_function
from collections import defaultdict
from collections.abc import Set


class TrieNode(Set):
    
    # initialization: adding the elements (tokens) to the set
    def __init__(self, iterable=()):
        self._children = defaultdict(TrieNode)
        self._end = False
        for element in iterable:
            self.add(element)
    
    # constructing the trie by adding each item (character/letter) of the element to a node and then moving to the next node
    def add(self, element):
        node = self
        for s in element:
            node = node._children[s]
        node._end = True
    
    # function on searching on an element and returning False or True if it exists in the trie; valuable for debugging
    def __contains__(self, element):
        node = self
        for k in element:
            if k not in node._children:
                return False
            node = node._children[k]
        return node._end
    
    # function on searching for the element and getting back with all possible results; valuable for debugging
    def search(self, term):
        results = set() 
        element = []    
        def _search(m, node, i):
            element.append(m)
            if i == len(term):
                if node._end:
                    results.add(''.join(element))
            elif term[i] == '#':
                for k, child in node._children.items():
                    _search(k, child, i + 1)
            elif term[i] in node._children:
                _search(term[i], node._children[term[i]], i + 1)
            element.pop()
        _search('', self, 0)
        return results
    
    # function with getting back with the most probable solution
    def predict(self, term):
        search_result = list(self.search(term))
        occurence_list=[]
        for element in search_result:
            occurence_list.append(word_frequency(element, 'en',wordlist='large'))
        
        if occurence_list:
            max_value_index=occurence_list.index(max(occurence_list))
            return search_result[max_value_index]
        else:
            return None

    # required by the set class (could be optimized for avoiding recursion)
    def __iter__(self):
        return iter(self.search('#'))
    
    # required by the set class (could be optimized by adding each node a count)
    def __len__(self):
        return sum(1 for _ in self)
    


Adding words to the trie.

In [8]:
my_trie = TrieNode()

for word in word_list:
    my_trie.add(word)
    

In [9]:
# adding the tokens from the training file to the trie

df = pd.read_csv('training_tokens.txt', sep=" ", header=None, names=["Original"])

training_text = df["Original"].str.cat(sep=" ").split(" ")

from collections import defaultdict
train_text_word_occurence = defaultdict(int)
for word in training_text:
    train_text_word_occurence[word]+=1

In [10]:
for word in training_text:
    my_trie.add(word)

Loading and running the trie on the corrupted text

In [11]:
df_corr = pd.read_csv('corrupted_tokens.txt', sep=" ", header=None, names=["Original"])

In [12]:
df_corr["predictions"] = df_corr["Original"].apply(my_trie.predict)

In [13]:
df_corr.head(15)

,Original,predictions
0,T#a#,That
1,n#t#,note
2,r#c#l#s,recalls
3,a#,as
4,e#p#r#e#c#,experience
5,T#e,The
6,p#s#e#g#r#,passengers
7,w#r#,were
8,s#n#,song
9,f#r,for


In [17]:
df_corr["predictions"].to_csv('recovered_tokens.txt', header=None, index=None, sep=' ', mode='a')

***

### Write-up and disclaimer ###

**Looking for solutions**:

There were a few solutions which came to my mind before choosing the trie implementation. 

**Splitting the tokens/words in pairs of two** and then iterating through them would also be a solution but in that case I would need to handle the odd words differently than the even ones and also the solution would not be so robust as it would not work with words having two (or more) wildcards beeing next to each other.

Finally I decided to use the **trie** as it seemed the best solution in this case.

However my solution is by far not the best. First of all I am sure that I could find a larger vocabulary (I used the one from the nltk) as there were a few words which were not in the vocabulary. Those - as you will see - are empty lines. Of course this could be also improved, just by taking a look at these and figuring them out, however that would probably lead to an overfit model. A better vocabulary would solve this problem. 

The model could be <u>improved by adding POS (part of speech tagging)</u> in case the words/tokens are in sentence order. Also another (or maybe additional) extension would be by adding a <u>word based n-grams</u> to the model. However the trainig of ngrams would be crutial as it could both overfit or underfit the text. 

As a self-check I trained the trie on the training tokens and and tried to predict them and reached a 86% accuracy. Of course this is an overfitted model, as the test text is probably not from the same text environment as the training text.


